In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

BASE_PATH = "C:/Users/mkasl/Desktop/donem5/yap470/Ara_rapor_ 1/final_dataset1"
TRAIN_PATH = os.path.join(BASE_PATH, 'train')
VALID_PATH = os.path.join(BASE_PATH, 'valid')
TEST_PATH = os.path.join(BASE_PATH, 'test')

IMAGE_SIZE = (224, 224)  #boyutunu arttırdım bu model için en iyi sonuç buymuş.
BATCH_SIZE = 32
tf.random.set_seed(42) #aynı sonucu vermesi için bunu gerçi dosyalara test train valid olarak ayırıyoruz
#tekrarlanabilirliği sağlıyor
INITIAL_EPOCHS = 10 #İlk eğitim aşaması için
FINE_TUNE_EPOCHS = 20 #Finetuning
#%70 %15 %15 dağılım
#Düzeltme hata alıyorum bu modele uygun hale getiriyorum -1 ile 1 arasında input bekliyor
#https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v2/preprocess_input
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
def load_and_prep_dataset(path, shuffle=False):
    dataset = tf.keras.utils.image_dataset_from_directory(
        path, labels='inferred', label_mode='categorical',
        image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, shuffle=shuffle, seed=42)
    #Bunu yapmazsam hata alıyorum bu modele uygun hale getiriyorum -1 ile 1 arasında input bekliyor
    dataset = dataset.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
#veri setini hazırlama
train_dataset = load_and_prep_dataset(TRAIN_PATH, shuffle=True)
validation_dataset = load_and_prep_dataset(VALID_PATH)
test_dataset = load_and_prep_dataset(TEST_PATH)
class_names = train_dataset.class_names
#https://www.tensorflow.org/api_docs/python/tf/keras/applications/MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
                                               include_top=False, #Sadece öznitelik çıkaran kısmını alıyoruz
                                               weights='imagenet') #Eğitilmiş ağırlıklarıyla birlikte
base_model.trainable = False #katmanları donduruyoruz 
#GlobalAveragePooling2D sınıflandırıcısı ekliyorum dondurulan modele
inputs = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)
model = tf.keras.Model(inputs, outputs, name="Transfer_Learning_Model")
model.summary()
#https://www.tensorflow.org/tutorials/images/transfer_learning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#üst katmanları eğitiyoruz ilk başta
history = model.fit(train_dataset,
                    epochs=INITIAL_EPOCHS,
                    validation_data=validation_dataset)

#finetuning aşaması
base_model.trainable = True #öğrenmesini açıyorum
#Önceden eğitilen model üzerine çok dokunamk istemediğimden çok düşük bir öğrenme oranı kullandım
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#history
history_fine_tune = model.fit(train_dataset,
                              epochs=INITIAL_EPOCHS + FINE_TUNE_EPOCHS,
                              initial_epoch=history.epoch[-1] + 1, #X eksenini dinamik olarak ayarlıyorum
                              validation_data=validation_dataset)

test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"\nTest Seti Doğruluğu: {test_accuracy:.4f}")
print(f"Test Seti Kaybı: {test_loss:.4f}")
# Karmaşıklık matrisi
#Test setindeki tüm resimler için tahminleri al
y_pred_probs = model.predict(test_dataset)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true_labels = np.concatenate([y for x, y in test_dataset], axis=0)
y_true = np.argmax(y_true_labels, axis=1)
conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Karmaşıklık Matrisi')
plt.show()